## City Learn

### 1. The Environment

CityLearn provides an environment similar to gym.Env with methods like:

- reset(): to initialize the simulation.

- step(actions): to perform an action for each building and return observations, rewards, done, and info.

It simulates:

- Multiple buildings

- Time steps (e.g., hourly for a whole year)

- Energy demand, PV generation, battery storage, DHW tank, etc.

### 2. State (Observation)
At each time step, each building observes:

- Outdoor temperature

- Solar generation

- Net electricity consumption

- Current electricity price

- Carbon intensity

- Storage level of battery and DHW

- Time features (hour, day, month...)

**These are the inputs to your RL agent (the "state").**

```python
pip install stable-baselines3[extra]
```

## Basic Workflow
Load schema (building configuration, weather, metadata):

In [1]:
import citylearn

print(citylearn.__version__)



2.0.0


In [3]:
from citylearn.agents.base import BaselineAgent as Agent
from citylearn.citylearn import CityLearnEnv

# initialize
env = CityLearnEnv('citylearn_challenge_2023_phase_2_local_evaluation', central_agent=True)
model = Agent(env)

# step through environment and apply agent actions
observations, _ = env.reset()

while not env.terminated:
    actions = model.predict(observations)
    observations, reward, info, terminated, truncated = env.step(actions)

# test
kpis = model.env.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value').round(3)
kpis = kpis.dropna(how='all')
display(kpis)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [13]:
# A tiny helper that always samples the space
def random_actions(action_spaces):
    return [space.sample() for space in action_spaces]

In [17]:
# Roll one complete episode

obs, info = env.reset()
while not env.terminated:                        # env.terminated becomes True at the end
    acts = random_actions(env.action_space)      # pure exploration!
    obs, reward, info, terminated, truncated = env.step(acts)

ValueError: too many values to unpack (expected 2)

### Smoke test to explore

In [20]:
from citylearn.citylearn import CityLearnEnv
import numpy as np
import matplotlib.pyplot as plt

# 1️⃣ Instantiate your env
env = CityLearnEnv('citylearn_challenge_2022_phase_1')

# 2️⃣ Reset and grab initial obs
obs = env.reset()

# 3️⃣ Prepare storage for district metrics
consumption_ts = []
cost_ts        = []

# 4️⃣ Loop over the fixed episode length (or break on done)
for step in range(env.episode_length):
    # Sample random actions
    if isinstance(env.action_space, list):
        acts = [space.sample() for space in env.action_space]
    else:
        acts = env.action_space.sample()

    # Step and unpack
    obs, reward, done, info = env.step(acts)

    # Record totals
    consumption_ts.append(np.sum(env.net_electricity_consumption))
    cost_ts.append(       np.sum(env.net_electricity_consumption_cost))

    if done:
        break

# 5️⃣ Convert to arrays
consumption = np.array(consumption_ts)
cost        = np.array(cost_ts)

# 6️⃣ Plot your results
plt.figure()
plt.plot(consumption, label='Net Consumption (kWh)')
plt.xlabel('Timestep')
plt.ylabel('District Net Electricity')
plt.legend()
plt.show()

plt.figure()
plt.plot(cost, label='Electricity Cost (€)')
plt.xlabel('Timestep')
plt.ylabel('District Cost')
plt.legend()
plt.show()


AttributeError: 'CityLearnEnv' object has no attribute 'episode_length'

1.4.4
